In [ ]:
from __future__ import print_function

import sys

sys.path.extend(['/home/jorg/repository/dcnn_acdc'])

import matplotlib
import matplotlib.pyplot as plt

from matplotlib import cm

from datetime import datetime
from pytz import timezone
import copy

import numpy as np
import torch

import numpy as np
import dill
import os
import glob
from collections import OrderedDict

from in_out.load_data import ACDC2017DataSet
from in_out.detector.detector_dataset import RegionDetectorDataSet
from config.config import config as config_acdc
from common.detector.config import config_detector
from plotting.color_maps import transparent_cmap
from utils.exper_hdl_ensemble import ExperHandlerEnsemble
from common.dslices.config import config as config_dslices
from common.detector.box_utils import find_bbox_object, BoundingBox
from utils.detector.batch_handler import BatchHandler
from utils.detector.experiment import Experiment
from utils.detector.exper_handler import ExperimentHandler

mycmap = transparent_cmap(plt.get_cmap('jet'))

%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
# Constants aka fixed variables
ROOT_DIR = "/home/jorg/repository/dcnn_acdc"

# set seeds
SEED = 4325
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.enabled = True
torch.backends.cudnn.deterministic = True
np.random.seed(SEED)

# Load our baseline 
seg_exper_ensemble = ExperHandlerEnsemble(config_dslices.exper_dict_brier)


In [ ]:
from in_out.detector.detector_dataset import create_dataset

dataset = create_dataset(seg_exper_ensemble, train_fold_id=0, quick_run=False, num_of_input_chnls=3,
                         model_name="rd1", type_of_map="e_map", use_raw_errors=True)

In [ ]:
batch_size = 8
train_batch = BatchHandler(dataset, is_train=True, cuda=False, keep_bounding_boxes=False, verbose=False)

In [ ]:
x_input, y_lbl_dict = train_batch(batch_size=128, do_balance=False, keep_batch=True)

In [ ]:
fold_id = 0
exper_handler_acdc = seg_exper_ensemble.seg_exper_handlers[fold_id]
train_batch.visualize_batch(grid_spacing=8, index_range=[113, 115], base_apex_only=False, sr_threshold=0.,
                               exper_handler=exper_handler_acdc)

In [ ]:
slice_num = 5185
print(dataset.train_patient_slice_id[slice_num])

input_channels = dataset.train_images[slice_num]
labels = dataset.train_labels[slice_num]
roi = dataset.train_lbl_rois[slice_num][0]
print(labels.shape)
print(roi)
roi_bbox = BoundingBox.create(roi)
print(roi_bbox.width, roi_bbox.height)
print(roi_bbox.area)
# slice(120, 200, None), slice(167, 247, None)
patch_labels = labels[120:200, 167:247]

fig = plt.figure(figsize=(16, 16))

ax1 = plt.subplot2grid((4, 4), (0, 0), rowspan=2, colspan=2)
ax1.imshow(labels)
roi_patch = roi_bbox.get_matplotlib_patch()
ax1.add_patch(roi_patch)

ax2 = plt.subplot2grid((4, 4), (0, 2), rowspan=2, colspan=2)
ax2.imshow(patch_labels)

plt.show()

In [ ]:
zeros = 0
negatives = 0
for slice_num, train_lbl_rois in enumerate(dataset.train_lbl_rois):
    if train_lbl_rois.shape[0] != 0:
        # print(train_lbl_rois)
        roi = train_lbl_rois[0]
        if np.any(roi < 0):
            negatives += 1
            print(slice_num)
        elif np.any(roi == 0):
            zeros += 1
        else:
            roi_box = BoundingBox.create(roi)
            if roi_box.area > 10000:
                print(slice_num)
print(zeros, negatives)